In [2]:
import utils
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack
import pandas as pd
from skrub import TableReport
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
import numpy as np

In [3]:
X, y, X_test = utils.get_and_process_data()

/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:197: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting val

In [9]:
import pandas as pd
from skrub import TableVectorizer


# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(X) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The TableVectorizer or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [8]:
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# Sanitize feature names
def sanitize_feature_names(df):
    """Sanitize feature names to remove special JSON characters."""
    df.columns = df.columns.str.replace(r"[^\w\d_]", "_", regex=True)
    return df

# Sanitize training and validation feature names
X_train_transformed = sanitize_feature_names(pd.DataFrame(X_train_transformed))
X_val_transformed = sanitize_feature_names(pd.DataFrame(X_val_transformed))

# Define the objective function for Optuna
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),  # Number of trees
        'max_depth': trial.suggest_int('max_depth', 2, 20),         # Maximum depth of trees
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),  # Min samples to split
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),    # Min samples per leaf
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),  # Max features
    }
    model = RandomForestRegressor(**param, random_state=42, n_jobs=-1)
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_val_transformed)
    rmse = mean_squared_error(y_val, y_pred)
    return rmse  # Minimize RMSE

# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')  # Minimizing RMSE
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params

# Save the best parameters to a file using joblib
joblib.dump(best_params, 'random_forest_best_params.pkl')

# Train the final model with the best parameters
X_transformed = sanitize_feature_names(pd.DataFrame(vectorizer.transform(X)))  # Transform the entire dataset
final_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_transformed, y)


[I 2024-12-10 18:01:01,181] A new study created in memory with name: no-name-64f497dd-b4b0-4a09-9550-1914464f1eb7
[I 2024-12-10 18:03:14,675] Trial 0 finished with value: 1.710837811764358 and parameters: {'n_estimators': 190, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_features': None}. Best is trial 0 with value: 1.710837811764358.
[I 2024-12-10 18:03:17,394] Trial 1 finished with value: 2.5415213007533013 and parameters: {'n_estimators': 107, 'max_depth': 2, 'min_samples_split': 11, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: 1.710837811764358.
[I 2024-12-10 18:03:40,317] Trial 2 finished with value: 1.621781685808711 and parameters: {'n_estimators': 169, 'max_depth': 16, 'min_samples_split': 9, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 2 with value: 1.621781685808711.
[I 2024-12-10 18:03:48,392] Trial 3 finished with value: 1.9215167540868334 and parameters: {'n_estimators': 107, 'max_depth': 10, 'min_sampl

RandomForestRegressor(max_depth=17, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=14, n_estimators=117, n_jobs=-1,
                      random_state=42)

In [ ]:
import optuna
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib
import logging
from tqdm import tqdm  # For progress bars

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Sanitize feature names
def sanitize_feature_names(df):
    """Sanitize feature names to remove special JSON characters."""
    logging.info("Sanitizing feature names.")
    df.columns = df.columns.str.replace(r"[^\w\d_]", "_", regex=True)
    return df

# Sanitize training and validation feature names
logging.info("Sanitizing training and validation feature names.")
X_train_transformed = sanitize_feature_names(pd.DataFrame(X_train_transformed))
X_val_transformed = sanitize_feature_names(pd.DataFrame(X_val_transformed))

# Define the objective function for Optuna
def objective(trial):
    logging.info(f"Starting trial {trial.number}")
    
    # Hyperparameters for polynomial regression
    degree = trial.suggest_int('degree', 2, 5)  # Degree of the polynomial
    logging.info(f"Trial {trial.number}: Using degree={degree}.")
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_transformed)
    X_val_poly = poly.transform(X_val_transformed)
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_poly)
    X_val_scaled = scaler.transform(X_val_poly)
    
    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    logging.info(f"Trial {trial.number}: Model trained.")
    
    # Predict and calculate RMSE
    y_pred = model.predict(X_val_scaled)
    rmse = mean_squared_error(y_val, y_pred, squared=False)  # Root Mean Squared Error
    logging.info(f"Trial {trial.number}: RMSE = {rmse:.4f}.")
    return rmse  # Minimize RMSE

# Create a study and optimize the objective function
logging.info("Starting Optuna study.")
study = optuna.create_study(direction='minimize')  # Minimizing RMSE
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params
logging.info(f"Best parameters: {best_params}")

# Save the best parameters to a file using joblib
joblib.dump(best_params, 'polynomial_regression_best_params.pkl')
logging.info("Best parameters saved to 'polynomial_regression_best_params.pkl'.")

# Train the final model with the best parameters
logging.info("Training final model.")
final_degree = best_params['degree']

# Final model training
poly = PolynomialFeatures(degree=final_degree, include_bias=False)
X_train_poly = poly.fit_transform(X_train_transformed)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)

final_model = LinearRegression()
final_model.fit(X_train_scaled, y_train)
logging.info("Final model trained.")


2024-12-11 10:05:45,983 - Sanitizing training and validation feature names.
2024-12-11 10:05:45,990 - Sanitizing feature names.
2024-12-11 10:05:46,001 - Sanitizing feature names.
2024-12-11 10:05:46,004 - Starting Optuna study.
[I 2024-12-11 10:05:46,008] A new study created in memory with name: no-name-c6c463bd-bacd-4dcd-b1c3-1d2d5ed23517
2024-12-11 10:05:46,014 - Starting trial 0
2024-12-11 10:05:46,019 - Trial 0: Using degree=4.
